In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

# Typography / plot style (legends are mechanism names only)
mpl.rc('font', family='serif', serif=['Times New Roman'])
mpl.rcParams.update({
    'text.usetex': False,
    'font.size': 16,
    'axes.titlesize': 20,
    'axes.labelsize': 20,
    'xtick.labelsize': 20,
    'ytick.labelsize': 20,
    'legend.fontsize': 20,
    'figure.titlesize': 20,
})
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 30

# --- your project utilities ---
from data_utils import correlated_data_generator, get_true_frequencies
from metrics import compute_mse

# --- baselines ---
from spl import random_split_perturb, random_split_estimate
from rs_fd import rs_fd_perturb, rs_fd_estimate
from rs_rfd import rs_rfd_perturb, rs_rfd_estimate

from corr_rr import (
    corr_rr_phase1_spl,
    corr_rr_phase2_perturb,
    corr_rr_estimate,
    combine_phase_estimates,
    optimal_p_y,
    build_p_y_table,   # now exists
)

# ---------------- helpers ----------------

def _normalize_dist(d):
    vals = np.array([max(0.0, float(v)) for v in d.values()], dtype=float)
    s = vals.sum()
    if s <= 0:
        k = len(vals)
        vals = np.full(k, 1.0 / k)
    else:
        vals = vals / s
    return {k: vals[i] for i, k in enumerate(d.keys())}

def _build_p_y_table_minimal(est_I, epsilon, n2, domain, cols):
    """
    Build p_y[(a->b)] for all ordered pairs using corr_rr.optimal_p_y().
    Signature kept identical to what you previously used.
    """
    return {
        (a, b): float(optimal_p_y(est_I[a], est_I[b], epsilon, n2, domain))
        for a in cols for b in cols if a != b
    }

# ---------------- one-shot runner ----------------

def run_all_once(
    df,
    epsilon,
    use_corr_rr=True,
    frac_phase1_corr=0.1,
    frac_phase1_rsrfd=0.1,   # Phase I fraction for RS+RFD two-phase prior learning
):
    """
    Compute per-mechanism MSEs on one dataset for a given ε.
    Returns dict with keys: "SPL", "RS+FD", "RS+RFD", optionally "Corr-RR".
    """
    cols = list(df.columns)
    domains = {c: sorted(df[c].unique().tolist()) for c in cols}
    true_freqs = get_true_frequencies(df, cols)

    out = {}

    # --- SPL ---
    rpt = random_split_perturb(df, epsilon)
    est = random_split_estimate(rpt, df, epsilon)
    out["SPL"] = np.mean([compute_mse(true_freqs[c], est[c]) for c in cols])

    # --- RS+FD ---
    pert_rsfd = rs_fd_perturb(df, domains, epsilon)
    est_rsfd = rs_fd_estimate(pert_rsfd, domains, epsilon)
    out["RS+FD"] = np.mean([compute_mse(true_freqs[c], est_rsfd[c]) for c in cols])

    # --- RS+RFD (Two-phase variant: Phase I used to learn priors; final = COMBINED) ---
    est_I_rsrfd, df_B_rsrfd, doms_rsrfd = corr_rr_phase1_spl(df, epsilon, frac=frac_phase1_rsrfd)
    n1_rsrfd = len(df) - len(df_B_rsrfd)
    n2_rsrfd = len(df_B_rsrfd)

    priors_2p = {col: _normalize_dist(est_I_rsrfd[col]) for col in est_I_rsrfd}
    pert_rsrfd_2p = rs_rfd_perturb(df_B_rsrfd, doms_rsrfd, priors_2p, epsilon)
    est_II_rsrfd = rs_rfd_estimate(pert_rsrfd_2p, doms_rsrfd, priors_2p, epsilon)

    # FINAL: count-weighted combine Phase I (SPL) and Phase II (RS+RFD)
    combined_rsrfd = combine_phase_estimates(est_I_rsrfd, est_II_rsrfd, n1_rsrfd, n2_rsrfd)
    out["RS+RFD"] = np.mean([compute_mse(true_freqs[c], combined_rsrfd[c]) for c in cols])

    # If you ever want the Phase-II-only variant again, just swap the line above for this one:
    # out["RS+RFD"] = np.mean([compute_mse(true_freqs[c], est_II_rsrfd[c]) for c in cols])

    # --- Corr-RR (optional) ---
    if use_corr_rr:
        est_I, df_B, doms_stable = corr_rr_phase1_spl(df, epsilon, frac=frac_phase1_corr)
        n1 = len(df) - len(df_B)
        n2 = len(df_B)
        common_domain = doms_stable[cols[0]]  # assumes common domain

        # Build p_y table
        p_y_table = _build_p_y_table_minimal(est_I, epsilon, n2, common_domain, cols)
        # (Alternatively: p_y_table = build_p_y_table(est_I, n2, doms_stable))

        # Phase II
        pert_corr = corr_rr_phase2_perturb(df_B, epsilon, est_I, doms_stable, p_y_table)

        # Estimation and combine
        est_II = corr_rr_estimate(pert_corr, doms_stable, epsilon)
        combined_est = combine_phase_estimates(est_I, est_II, n1, n2)
        out["Corr-RR"] = np.mean([compute_mse(true_freqs[c], combined_est[c]) for c in cols])

    return out

# ---------------- sweep & plot ----------------

def sweep_all(
    n=10000,
    epsilons=(0.2, 0.4, 0.6, 0.8, 1.0),
    R=50,
    corr=0.9,
    plot_dir=None,
    csv_dir=None,
    seed=None,
    use_corr_rr=True,
    frac_phase1_corr=0.1,
    frac_phase1_rsrfd=0.1,
):
    """
    Run R trials for each ε, average MSEs, and plot.
    RS+RFD is the TWO-PHASE variant. Corr-RR optional.
    """
    domain = [0, 1,2,3,4,5]
    correlations = [('X1', 'X2', corr)]
    keys = ["SPL", "RS+FD", "RS+RFD"] + (["Corr-RR"] if use_corr_rr else [])
    means = {k: np.zeros(len(epsilons)) for k in keys}

    if seed is not None:
        np.random.seed(seed)

    attr_count = None
    domain_size = len(domain)

    for run in range(R):
        df = correlated_data_generator(
            domain,
            n=n,
            correlations=correlations,
            total_attributes=2,
            seed=None
        )
        if attr_count is None:
            attr_count = df.shape[1]

        for j, eps in enumerate(epsilons):
            res = run_all_once(
                df,
                eps,
                use_corr_rr=use_corr_rr,
                frac_phase1_corr=frac_phase1_corr,
                frac_phase1_rsrfd=frac_phase1_rsrfd,
            )
            for k in keys:
                means[k][j] += res[k]

    for k in keys:
        means[k] /= R

    # Plot
    plt.figure(figsize=(10, 8))
    plt.plot(epsilons, means["SPL"],         '-o', linewidth=3, markersize=16, label='SPL')
    plt.plot(epsilons, means["RS+FD"],       '-s', linewidth=3, markersize=16, label='RS+FD')
    plt.plot(epsilons, means["RS+RFD"],      '-^', linewidth=3, markersize=16, label='RS+RFD')
    if use_corr_rr:
        plt.plot(epsilons, means["Corr-RR"], '-D', linewidth=3, markersize=16, label='Corr-RR')

    plt.xlabel(r'$ϵ$', fontsize=50)
    plt.ylabel('MSE',  fontsize=40)
    plt.xticks(epsilons, labels=[str(e) for e in epsilons])
    plt.tick_params(axis='both', which='major', labelsize=30)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0))
    plt.legend(fontsize=35, loc='upper right', frameon=True, edgecolor='black')
    plt.tight_layout()

    # (Optional) print to inspect
    #if use_corr_rr: print(means["Corr-RR"])

    # Output filenames
    def _fmt(x): return f"{x:g}"
    base = f"mseVSepsilon_{attr_count}attribute_{domain_size}domain_n_{n}_frac_{_fmt(frac_phase1_rsrfd)}_corr_{_fmt(corr)}"

    # Save PDF
    if plot_dir:
        os.makedirs(plot_dir, exist_ok=True)
        pdf_path = os.path.join(plot_dir, base + ".pdf")
        plt.savefig(pdf_path, format="pdf")

    plt.show()

    # Save CSV
    if csv_dir:
        os.makedirs(csv_dir, exist_ok=True)
        df_out = pd.DataFrame({"epsilon": list(epsilons)})
        for k in keys:
            df_out[k] = means[k]
        csv_path = os.path.join(csv_dir, base + ".csv")
        df_out.to_csv(csv_path, index=False)

    return means




In [19]:
# ---------------- p_y table viewer ----------------



def optimal_p_y(f_a, f_b, epsilon, n, domain):
    """
    Closed-form minimizer of average MSE from your Proposition.
    epsilon is unused in this closed-form; kept for API compatibility.
    """
    k = len(domain)
    n2 = float(n)
    num = 0.0
    den = 0.0
    for v in domain:
        fa = float(f_a.get(v, 0.0))
        fb = float(f_b.get(v, 0.0))
        e  = 2.0*fb - 1.0
        d0 = 1.0 - fa - fb
        a0 = fa - fb
        num += (d0*e)/(2.0*k) - (a0*e)/(2.0*n2*k)
        den += (e*e)*((1.0/(4.0*k)) - (1.0/(4.0*n2*k)))
    if abs(den) < 1e-12:
        return 0.5
    return float(np.clip(num/den, 0.0, 1.0))

def p_y_tables_for_epsilons(
    df,
    epsilons,
    frac_phase1_corr=0.1,
    use_minimal_builder=True,
    csv_dir=None,
    float_fmt="%.6f",
):
    """
    For each epsilon:
      - run Phase I via corr_rr_phase1_spl
      - build p_y[(a->b)] using either _build_p_y_table_minimal(...) or build_p_y_table(...)
      - pretty-print and (optionally) save a CSV matrix with rows a, cols b

    Returns: dict {epsilon: pandas.DataFrame} where DataFrame is the p_y matrix.
    """
    import pandas as pd
    os.makedirs(csv_dir, exist_ok=True) if csv_dir else None

    cols = list(df.columns)
    results = {}

    for eps in epsilons:
        # Phase I (SPL) to get est_I and stable domains
        est_I, df_B, doms_stable = corr_rr_phase1_spl(df, eps, frac=frac_phase1_corr)

        # choose builder
        if use_minimal_builder:
            # assumes a common domain (as in your Corr-RR experiments)
            common_domain = doms_stable[cols[0]]
            pmap = _build_p_y_table_minimal(est_I, eps, len(df_B), common_domain, cols)
        else:
            # your full builder (if it handles mixed domains)
            pmap = build_p_y_table(est_I, len(df_B), doms_stable)

        # pivot to a matrix with NaN on diagonal
        mat = pd.DataFrame(index=cols, columns=cols, dtype=float)
        for a in cols:
            for b in cols:
                if a == b:
                    mat.loc[a, b] = float('nan')
                else:
                    mat.loc[a, b] = float(pmap[(a, b)])

        # pretty print
        print(f"\n=== p_y table (Corr-RR Phase I → optimization) for epsilon = {eps} ===")
        with pd.option_context('display.float_format', lambda v: float_fmt % v):
            print(mat)

        # optional save
        if csv_dir:
            fname = os.path.join(csv_dir, f"py_table_eps_{str(eps).replace('.', '_')}.csv")
            mat.to_csv(fname, float_format=float_fmt)
            print(f"[saved] {fname}")

        results[eps] = mat

    return results


# ---------------- example: run just the p_y tables ----------------
if __name__ == "__main__":
    # build ONE dataset (fixed across eps) so differences are due to epsilon, not resampling
    domain = [0, 1,2,3,4]
    correlations = [('X1', 'X2', 0.99)]  # same default you used
    df = correlated_data_generator(
        domain,
        n=20000,
        correlations=correlations,
        total_attributes=2,
        seed=45,  # fix seed for reproducibility
    )

    eps_list = [0.2, 0.4, 0.6, 0.8, 1.0]

    _ = p_y_tables_for_epsilons(
        df,
        eps_list,
        frac_phase1_corr=0.2,       # match your sweep args if you want
        use_minimal_builder=True,   # uses optimal_p_y(est_I[a], est_I[b], ...)
        #csv_dir=r"C:\Users\ss6365\Desktop\pet_final\data\py_tables"  # or None to skip saving
    )



=== p_y table (Corr-RR Phase I → optimization) for epsilon = 0.2 ===
         X1       X2
X1      NaN 0.000000
X2 0.000000      NaN

=== p_y table (Corr-RR Phase I → optimization) for epsilon = 0.4 ===
         X1       X2
X1      NaN 0.000000
X2 0.000000      NaN

=== p_y table (Corr-RR Phase I → optimization) for epsilon = 0.6 ===
         X1       X2
X1      NaN 0.000000
X2 0.000000      NaN

=== p_y table (Corr-RR Phase I → optimization) for epsilon = 0.8 ===
         X1       X2
X1      NaN 0.000000
X2 0.000000      NaN

=== p_y table (Corr-RR Phase I → optimization) for epsilon = 1.0 ===
         X1       X2
X1      NaN 0.000000
X2 0.000000      NaN
